In [1]:
import dask.dataframe as dd
from datetime import datetime, timedelta
exec(open("./lib/splitter.py").read())
exec(open("./lib/distance.py").read())
exec(open("./lib/tzMap.py").read())

In [2]:
startDate = '2019-12-18'
endDate = '2019-12-30'
outputDir = './data/splitted' + '/'
outputPrefix = 'out_hid_'
inputFile = './data/Santa_Clara_County_Pin_Report_1e6.tsv'

flushThresholdPerDate = 1000 # We will cache rows for a date in memory before writing to the files.


In [3]:
df = dd.read_csv(inputFile, sep='\t')

In [4]:
df.head(5)

,Polygon ID,Hashed Device ID,Lat of Visit,Lon of Visit,Unix Timestamp of Visit,Date,Time of Day,Day of Week,Time Zone
0,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.900276,-121.581208,1576733932,2019-12-18,21:38:52,Wed,America/Los_Angeles
1,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.898245,-121.574142,1576733977,2019-12-18,21:39:37,Wed,America/Los_Angeles
2,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.897755,-121.565889,1576734022,2019-12-18,21:40:22,Wed,America/Los_Angeles
3,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.900589,-121.560763,1576734067,2019-12-18,21:41:07,Wed,America/Los_Angeles
4,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.906981,-121.559551,1576734113,2019-12-18,21:41:53,Wed,America/Los_Angeles


In [5]:
df = df.rename(columns=columnNamesMap)
addGridColumnsDD(df)
df.head(5)


,pid,hid,lat,lon,ts,date,td,dw,tz,gridX,gridY,gridId
0,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.900276,-121.581208,1576733932,2019-12-18,21:38:52,Wed,America/Los_Angeles,3003,9188,300309188
1,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.898245,-121.574142,1576733977,2019-12-18,21:39:37,Wed,America/Los_Angeles,2982,9259,298209259
2,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.897755,-121.565889,1576734022,2019-12-18,21:40:22,Wed,America/Los_Angeles,2978,9341,297809341
3,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.900589,-121.560763,1576734067,2019-12-18,21:41:07,Wed,America/Los_Angeles,3006,9392,300609392
4,Santa Clara County,8880011d4969dafb2bff89d4da1a37c994292941,36.906981,-121.559551,1576734113,2019-12-18,21:41:53,Wed,America/Los_Angeles,3070,9404,307009404


In [6]:
# task 1: generate string dates from start and end

dates = getDateStrings(startDate, endDate)
initFilesWithGridCols(dates)

# we will caches rows for each hid until flushed to files. keys: hids, values: list of rows
cachedRows = {}
flushSize = 100000 # write after parsing flushSize number of rows.
progress = 0

count = 0 # to track number of rows in cache   
for index, row in df.iterrows():

    if (row['date'] in dates):
        progress = progress + 1
        
        row['tz'] = getTimeZoneId(row['tz'])
        
        if row['hid'] not in cachedRows:
            cachedRows[row['hid']] = []
        cachedRows[row['hid']].append(row)
        
    count += 1    
    if count >= flushSize:
        flushDataToFilesWithGrid(cachedRows)
        print(f"wrote {progress} lines.")
        count = 0

flushDataToFilesWithGrid(cachedRows, force=True)
print(f"wrote {progress} lines.")
        


KeyboardInterrupt: 